In [1]:
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
import geoplot
from shapely.geometry import Point
import pandas as pd
from stellargraph import StellarGraph

/home/asia/.pyenv/versions/3.8.0/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
2022-02-04 11:41:23.984281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-04 11:41:23.984302: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-04 11:41:25.908041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-04 11:41:25.909100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load

In [ ]:
G = nx.read_gpickle("data/bike_wroc.gpickle")

In [57]:
bike_df = gpd.read_file("dane/bike_network.gpkg")
bike_df=bike_df.set_crs(epsg=4326)

In [58]:
bike_df

,osmid,y,x,street_count,highway,ref,geometry
0,95584835,51.083111,17.049513,4,,,POINT (17.04951 51.08311)
1,95584841,51.084699,17.064367,3,,,POINT (17.06437 51.08470)
2,95584850,51.083328,17.035057,4,,,POINT (17.03506 51.08333)
3,95584852,51.083009,17.053573,4,,,POINT (17.05357 51.08301)
4,95584855,51.083333,17.034685,4,,,POINT (17.03468 51.08333)
...,...,...,...,...,...,...,...
50127,9368128601,51.144830,16.859986,3,,,POINT (16.85999 51.14483)
50128,9368128604,51.144997,16.859562,3,,,POINT (16.85956 51.14500)
50129,9368128611,51.146156,16.858962,3,,,POINT (16.85896 51.14616)
50130,9368261919,51.144914,16.859713,3,,,POINT (16.85971 51.14491)


In [98]:
investmap_df = gpd.read_file("dane/investmap.csv")
investmap_df["Longitude"]=investmap_df["Longitude"].apply(lambda x: float(x))
investmap_df["Latitude"]=investmap_df["Latitude"].apply(lambda x: float(x))
investmap_df["geometry"] = gpd.points_from_xy(investmap_df["Longitude"], investmap_df["Latitude"])
investmap_df=investmap_df.set_crs(epsg=4326)
investmap_df


,Id,Updated at,Latitude,Longitude,Title,Url,geometry
0,11,2016-11-04 13:10:00,51.039500,16.969200,Aura Bielany,"/inwestycja/aura-bielany,11.html",POINT (16.96920 51.03950)
1,12,2016-11-04 13:10:00,51.044300,16.967400,Quart Bielany II,"/inwestycja/quart-bielany-ii,12.html",POINT (16.96740 51.04430)
2,13,2016-11-04 13:10:00,51.034000,16.959900,"Osiedle domów jednorodzinnych, ul. Porzeczkowa",/inwestycja/osiedle-domow-jednorodzinnych-ul-p...,POINT (16.95990 51.03400)
3,14,2016-11-04 13:10:00,50.861500,17.456300,Kamienica Pod Lipą,"/inwestycja/kamienica-pod-lipa,14.html",POINT (17.45630 50.86150)
4,15,2016-11-04 13:10:00,50.862500,17.457600,Kamienica Pod Modrzewiem,"/inwestycja/kamienica-pod-modrzewiem,15.html",POINT (17.45760 50.86250)
...,...,...,...,...,...,...,...
5909,15361,2022-01-03 08:59:11,54.268526,16.086431,Zespół dwóch budynków w zabudowie wielorodzinn...,/inwestycja/zespol-dwoch-budynkow-w-zabudowie-...,POINT (16.08643 54.26853)
5910,15362,2022-01-03 09:10:20,54.264785,16.075423,Villa Verona Mielno,"/inwestycja/villa-verona-mielno,15362.html",POINT (16.07542 54.26479)
5911,15367,2022-01-06 14:37:12,50.095544,20.062668,Na Wzgórzach 29,"/inwestycja/na-wzgorzach-29,15367.html",POINT (20.06267 50.09554)
5912,15368,2022-01-08 16:56:57,52.205884,21.074588,Aparté Mokotów,"/inwestycja/aparte-mokotow,15368.html",POINT (21.07459 52.20588)


In [125]:
i=investmap_df.to_crs(epsg=2177) #Poland epsg
p=i.loc[0]["geometry"]
b=bike_df.to_crs(epsg=2177)
b.distance(p).min()

1132.5690028411534

In [135]:
G = nx.read_gpickle("dane/bike_wroc.gpickle")

In [ ]:
for n in G.nodes:
    G.nodes[n]["type"]="crossing"
G.nodes.data()

In [ ]:
nx.set_edge_attributes(G, "bicycle_path", "type")


In [138]:
for p_ind,p in i.iterrows():
    distance=b.distance(p["geometry"])
    idx=distance.idxmin()
    min_d=distance.min()
    node=f'{p["Id"]}_building'
    G.add_nodes_from([(node, {'type': 'building', 'name':p["Title"], 'y':p["Latitude"], 'x':p["Longitude"]})])
    G.add_edge(bike_df.iloc[idx]["osmid"], node, length=min_d, type="closest_crossing")
    

In [82]:
i.crs

<Derived Projected CRS: EPSG:2177>
Name: ETRF2000-PL / CS2000/18
Axis Info [cartesian]:
- x[north]: Northing (metre)
- y[east]: Easting (metre)
Area of Use:
- name: Poland - onshore and offshore between 16°30'E and 19°30'E.
- bounds: (16.5, 49.39, 19.5, 55.93)
Coordinate Operation:
- name: Poland CS2000 zone 6
- method: Transverse Mercator
Datum: ETRF2000 Poland
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
G.nodes.data()

In [ ]:
G.edges.data()

In [273]:
G=G.to_undirected()
nx.write_gpickle(G, "bike_and_investmap.gpickle")